In [47]:
from pymongo import MongoClient
import pandas as pd

## Assemble data and mount dataframe

First we need to get all the data from mongodb and mount in a dataframe so its easier to work with

In [48]:
# Creating connection to the DB
conn = MongoClient('localhost', 27017)
db = conn.nbascrapper
players_collection = db.players
players_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nbascrapper'), 'players')

In [49]:
dfs = []
# Unpack json array to df
def json_to_df(row, json_col):
    try:
        json_df = pd.DataFrame(row[json_col])
        dfs.append(json_df.assign(**row.drop(json_col)))
    except:
        print(row[json_col])

In [63]:
# Getting all 'Career Regular Season Stats' from players
query_filter = { 
        "stats": 
        {
            "$not":
            {
                "$eq":{}
            }
        },
        "stats.Career Regular Season Stats": 
        {
                "$exists": True
        }
    }

query_project = {
    '_id': 0,
    'id': 1,
    'position': 1,
    'stats.Career Regular Season Stats': 1
}

data = list(players_collection.find(query_filter, query_project))

df_unpacked = pd.io.json.json_normalize(data)
df_unpacked.apply(json_to_df, axis=1, json_col='stats.Career Regular Season Stats')

players = pd.concat(dfs)
players.set_index('id', inplace=True)

nan
nan


/media/lucas/DATA/Repos/nba_scrapper/venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## Analysing dataframe

In [64]:
print(f'Nº Players: {len(players)}')
print(f'Nº Columns: {len(list(players))}')
print(f'Columns: {list(players)}')
players.head()

Nº Players: 55104
Nº Columns: 59
Columns: ['%3PA', '%3PM', '%AST', '%BLK', '%BLKA', '%DREB', '%FGA', '%FGA  2PT', '%FGA  3PT', '%FGM', '%FTA', '%FTM', '%OREB', '%PF', '%PFD', '%PTS', '%PTS  2PT', '%PTS  2PT\xa0MR', '%PTS  3PT', '%PTS  FBPs', '%PTS  FT', '%PTS  OffTO', '%PTS  PITP', '%REB', '%STL', '%TOV', '2FGM  %AST', '2FGM  %UAST', '3FGM  %AST', '3FGM  %UAST', '3P%', '3PA', '3PM', 'AGE', 'AST', 'BLK', 'By Year', 'DREB', 'FG%', 'FGA', 'FGM', 'FGM  %AST', 'FGM  %UAST', 'FT%', 'FTA', 'FTM', 'GP', 'GS', 'MIN', 'OREB', 'PF', 'PTS', 'REB', 'STL', 'Season', 'TEAM', 'TOV', 'USG%', 'position']


,%3PA,%3PM,%AST,%BLK,%BLKA,%DREB,%FGA,%FGA 2PT,%FGA 3PT,%FGM,...,OREB,PF,PTS,REB,STL,Season,TEAM,TOV,USG%,position
id,,,,,,,,,,,,,,,,,,,,,
76001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,2,5,2.1,0.3,1994-95,SAC,0.8,NaN,F
76001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0.7,0.7,2.7,0,1994-95,PHL,1.7,NaN,F
76001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,1.9,4.7,2.1,0.3,1994-95,TOT,0.8,NaN,F
76001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,1.5,4.9,3.5,0.2,1993-94,BOS,1.3,NaN,F
76001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,2,5.3,3.1,0.5,1992-93,MIL,1.1,NaN,F


#### Columns description (As described in https://stats.nba.com/help/glossary/)

In [69]:
query_project = {
    '_id': 0
}

glossary_collection = db.glossary
glossary = pd.DataFrame(list(glossary_collection.find({},query_project)))
glossary.set_index('Key', inplace=True)
glossary.head()

,Name,Definition,Type,Contexts,Formula
Key,,,,,
%3PA,Percent of Team's 3 Point Field Goals Attempted,The percentage of a team's 3 point field goals...,Usage,- Usage - Clutch - Player -,NaN
%3PM,Percent of Team's 3 Point Field Goals Made,The percentage of a team's 3 point field goals...,Usage,- Usage - Clutch - Player -,NaN
%AST,Percent of Team's Assists,The percentage of a team's assists that a play...,Usage,- Usage - Clutch - Tracking - Player -,NaN
%BLK,Percent of Team's Blocks,The percentage of a team's blocks that a playe...,Usage,- Usage - Clutch - Defensive - Player -,NaN
%BLKA,Percent of Team's Blocked Field Goal Attempts,The percentage of a team's own blocked field g...,Usage,- Usage - Clutch - Player -,NaN
